In [ ]:
import pandas as pd
from textblob import TextBlob
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from ast import literal_eval
import nltk

import statsmodels.formula.api as smf # A convenience interface for specifying models using formula strings and DataFrames. Canonically imported using import statsmodels.formula.api as smf

# package for plotting graphs
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.colors as clrs

# Import packages

In [21]:
#nltk.download('brown')
# nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\laudi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [7]:
import pandas as pd
from textblob import TextBlob
import pickle
import numpy as np
import nltk
import spacy
import re
import os 
from ast import literal_eval

## Feature engineering

In [8]:
PATH_FIND = re.compile(r".*repository.")
rep_path = re.search(PATH_FIND,os.getcwd()).group()

In [12]:
raw_file = pd.read_csv(rep_path+"1 csv\\patent_data_RAW.csv", sep=",")

#### Noun creation

In [16]:
#define useful data in the dataset
patent_data = raw_file[["id", "issue_date", "abstract", "xi_real", 'subsection_id']]

In [ ]:
# defining functions
def extract_nouns(abstract):
    blob=TextBlob(abstract)
    nouns = list(blob.noun_phrases)
    return nouns

# Apply functions to the dataframe
patent_data['extracted_nouns'] = patent_data['abstract'].apply(extract_nouns)

#### Year creation 

In [ ]:
#extract years from the issue date
patent_data['issue_year'] = patent_data['issue_date'].str.slice(6,10)

In [ ]:
patent_data.to_csv("final_dataframe.csv")

#### Varible of reference

In [ ]:
#create list of years
years = sorted(list(set(patent_data['issue_year'])))

#extract subsections
patent_subsections = sorted(list(set(patent_data['subsection_id'])))

## Extract most used nouns

In [5]:
top_nouns_dict = {}
for i in patent_subsections: 
    top_nouns_dict_by_year={} 
    for y in years:
        combination=[]
        for m in patent_data['extracted_nouns'][(patent_data['issue_year']==y) & (patent_data['subsection_id']==i)]:
            for w in m: 
                combination.append(w)
        top_nouns_dict_by_year[y] = combination
    top_nouns_dict[i]= top_nouns_dict_by_year

In [ ]:
# we could clean a bit the results before proceeding
# this because not all nouns are good. 

In [6]:
def sort_extract(my_list, n = 10):
    """
    my_list -> list. We pass all the found nouns for combination year subsectionID
    n -> int. We decide the number of nouns 
    """
    
    # Then we extract top n most repeated words
    my_dict = {}
    for x in my_list:
        if x in my_dict.keys():
            my_dict[x] += 1
        else:
            my_dict[x] = 1
    sorted_dict = {key: value for (key, value) in sorted(my_dict.items(), key=lambda x: x[1], reverse=True)}
    top_noun = list(sorted_dict)[:n]
    return top_noun




In [ ]:
for i in patent_subsections:
    for y in years: 
        top_nouns_dict[i][y] = sort_extract(top_nouns_dict[i][y])

In [7]:
# export Dict
with open('top_nouns_dict.pkl', 'wb') as f:
    pickle.dump(top_nouns_dict, f)